## Condição de Corrida e Seção Crítica
Em sistemas de computação, a condição de corrida ocorre quando duas ou mais operações devem ocorrer na mesma ordem, mas a ordem em que as operações são executadas não é controlada pelo programa, o que resulta em comportamentos inesperados e/ou indeterminados. Estas condições são particularmente comuns in sistemas paralelos e concorrentes, como sistemas distribuídos e sistemas multithread.

Devido a sua natureza não determinística é uma situação difícil de prever. A condição de corrida é frequentemente considerada um dos problemas mais difíceis na programação concorrente. Estas condições podem causar problemas, como falhas de segurança, corrupção de dados e comportamentos de sistemas erráticos.

Para obter uma melhor compreensão das condições de corrida, vamos examinar un exemplo clássico de um sistema multithread: Imagine que temos duas threads, A e B, que precisam incrementar o valor armazenado em uma variável compartilhada.

A operação de incremento normalmente é composta por três passos:

1. Ler o valor atual da variável (por exemplo, counter é 0)
2. Incrementar o valor (por exemplo, 0 torna-se 1)
3. Gravar o valor incrementado de volta à variável

No entanto, em um sistema multithread sem controle adequado de acesso, a sequência dessas operações pode ser interrompida, como abaixo:

1. A thread A lê o valor atual do counter (0)
2. A thread B lê o valor atual do counter (ainda 0, pois A ainda não incrementou o valor)
3. A thread A incrementa o valor e grava o valor incrementado de volta (1)
4. A thread B, que ainda pensa que o valor é 0, incrementa o valor e grava o valor incrementado de volta (1)

Nesta situação, embora A e B tenham tentado aumentar a contagem, a contagem é apenas 1 em vez de 2. Isso é uma situação de competição entre os threads e é um bom exemplo de uma condição de corrida.

Para evitar condições de corrida, os programadores usam várias técnicas de controle de concorrência, como bloqueios (por exemplo, mutexes ou semáforos), que garantem que certas operações sejam atômicas, ou seja, indivisíveis. No contexto do exemplo acima, um semáforo pode ser usado para garantir que apenas uma thread possa realizar a operação de incremento de uma vez. Desta forma, a sequência de ler, incrementar e gravar é sempre concluída por uma thread antes que a outra comece, evitando a condição de corrida.

Em termos mais gerais, o conceito de 'seção crítica' é usado para denotar um bloco de código que acessa recursos compartilhados e que não deve ser simultaneamente executado por mais de um thread. As técnicas de controle de concorrência visam garantir que apenas um thread possa entrar em sua seção crítica de cada vez.


## Vamos ver um Exemplo

In [ ]:
import threading

# um objeto compartilhado
counter = 0

# função que será executada pelas threads
def increment_counter(thread_name):
    global counter
    for _ in range(5):  # reduzindo o número de operações para simplificar a visualização
        print(f"{thread_name} está incrementando o contador.")
        counter += 1
        print(f"{thread_name} incrementou o contador. Valor do contador: {counter}")

# criação de 3 threads
threads = []
for i in range(3):
    t = threading.Thread(target=increment_counter, args=(f"Thread {i + 1}",))
    threads.append(t)
    t.start()

# espera todas as threads terminarem
for t in threads:
    t.join()

print(f"\nValor final do contador: {counter}")
print("Valor esperado do contador: 15")  # 3 threads * 5 incrementos cada = 15



## O problema
No código acima deveríamos ter uma condição de corrida, não há nada que garanta que um thread não vá sobrescrever o valor do outro. Contudo, é python, então o GIL  (Global Interpreter Lock) não deixa o código CPU-Bond ser executado em paralelo, logo não há sobreposição de escrita no objeto compartilhado (Global) counter.

## solucionando nada.
Como não há problema, por causa do GIL, a criação do semáforo é meramente didática.

## Agora vamos forçar um problema de condição de corrida
Na maioria das linguagens, a condição de corrida é mais propensa a ocorrer quando temos uma operação que envolve mais de um passo e que é interrompida entre esses passos. Vamos tentar dividindo o incremento em duas etapas: uma para leitura e outra para gravação.

## Finalmente, o problema
Neste exemplo, conseguimos provocar uma condição de corrida. Novamente, só para lembrar, se fosse o java, ou o C++, talvez não precisássemos dividir a tarefa do thread em dois tempos. Como não há nenhuma restrição no compilador destas duas linguagens (ou interpretador) que é o caso do java, então existe uma chance de que dois threads tentem escrever ao mesmo tempo.

## Agora vamos corrigir de verdade.
Vou criar o semáforo, ativar e desativar nos lugares certos para garantir que não exista a condição de corrida.

In [ ]:
import threading
import time

# Inicializando o objeto compartilhado
counter = 0
# Criando um semáforo. Um semáforo é usado para controlar o acesso a um recurso comum.
semaphore = threading.Semaphore()

def increment_counter(thread_name):
    global counter
    for _ in range(3):
        # Cada thread deve adquirir o semáforo antes de poder ler/modificar o contador
        semaphore.acquire()

        print(f"{thread_name} está lendo o contador.")
        value = counter

        print(f"{thread_name} incrementará o contador.")
        time.sleep(0.1)  # Simulando algum atraso

        counter = value + 1
        print(f"{thread_name} incrementou o contador. Valor do contador: {counter}")

        # Depois que a thread terminou de modificar o contador, ela deve liberar o semáforo.
        # Isso permite que outras threads adquiram o semáforo e modifiquem o contador.
        semaphore.release()

# Criando as threads
threads = []
for i in range(3):
    # Criando uma nova thread
    t = threading.Thread(target=increment_counter, args=(f"Thread {i + 1}",))
    threads.append(t)
    t.start()  # Iniciando a thread

# Esperando todas as threads terminarem antes de mostrar o valor final do contador
for t in threads:
    t.join()

print(f"\nValor final do contador: {counter}")
print("Valor esperado do contador: 9")  # 3 threads * 3 incrementos cada = 9


## Um exemplo de como usar a Beautifullsoap

In [ ]:
# Importando as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup

# A URL da página da web que você quer acessar
url = "https://en.wikipedia.org/wiki/Race_condition#:~:text=A%20race%20condition%20can%20arise,bugs%20due%20to%20unanticipated%20behavior"

# Enviar uma requisição GET para a página da web
response = requests.get(url)

# Inicializar um objeto BeautifulSoup com o conteúdo da página
soup = BeautifulSoup(response.content, 'html.parser')

# Extrair todo o texto da página (sem tags HTML) e armazená-lo em uma variável
page_text = soup.get_text()

# Imprimir o texto da página
print(page_text)


# Atividade Prática Avaliativa Em Grupo - RA3-1
Para realizar esta atividade e conseguir os pontos equivalentes na média da RA-3 você deve fazer uma cópia deste notebook, manter inalterado este enunciado, incluir o nome de todos os componentes do grupo e, finalmente resolver o exercício a seguir em uma célula de código:

Seu objetivo é escrever um programa Python que recupere o texto de cinco páginas diferentes da Wikipedia. Cada página deve ser recuperada por uma thread diferente.

O programa deve cumprir os seguintes requisitos:

1. Use a biblioteca requests para realizar requisições HTTP GET para as páginas da Wikipedia.
2. Use a biblioteca BeautifulSoup para extrair o texto das páginas da web.
3. Crie uma thread para cada página da Wikipedia que você está acessando.
4. Cada thread deve escrever o texto extraído no mesmo arquivo de texto compartilhado.
5. Garanta que não ocorram condições de corrida ao escrever no arquivo de texto compartilhado.
6. O thread que gravou cada texto no arquivo deve ser identificado no próprio texto gravado e em uma impressão no terminal que permita acompanhar o processo de gravação.

Lembre-se não podem haver condições de corrida. Use semáforos.


In [ ]:
#Grupo: Andrey Bonat, Bruno Miglioretto, Vítor Vale

import threading
import requests
from bs4 import BeautifulSoup
import time

urls = ["https://pt.wikipedia.org/wiki/Capivara",
       "https://pt.wikipedia.org/wiki/Marvel_Comics",
       "https://pt.wikipedia.org/wiki/Ci%C3%AAncia_da_computa%C3%A7%C3%A3o",
       "https://pt.wikipedia.org/wiki/Wikip%C3%A9dia",
       "https://pt.wikipedia.org/wiki/Cultura_da_Regi%C3%A3o_Nordeste_do_Brasil"]
counter = 0
file = open("textos.txt", "a")
semaphore = threading.Semaphore()

def ExtrairTexto(url, i):
    global file
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    page_text = soup.get_text()
    separator = "------------------------------------------------" + str(i) + "------------------------------------------------\n"
    semaphore.acquire()
    file.write(separator)
    print("Id Thread: ", i)
    file.write(page_text)
    semaphore.release()

threads = []
for i,url in enumerate(urls):
    t = threading.Thread(target=ExtrairTexto, args=(url,i))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

file.close()

Id Thread:  4
Id Thread:  1
Id Thread:  2
Id Thread:  0
Id Thread:  3
1.5246961116790771
